<a href="https://colab.research.google.com/github/9-coding/Google_ML_Bootcamp_5th/blob/main/NL2SQL_gemma2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
!pip install --quiet datasets
!pip install --quiet huggingface_hub

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGr

In [3]:
import pandas as pd
from datasets import load_dataset

# Load Dataset

In [19]:
dataset = load_dataset("Clinton/Text-to-sql-v1")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text'],
        num_rows: 262208
    })
})

In [20]:
dataset['train'][0]

{'instruction': 'Name the home team for carlton away team',
 'input': 'CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n)',
 'response': 'SELECT home_team FROM table_name_77 WHERE away_team = "carlton"',
 'source': 'sql_create_context',
 'text': 'Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables. ### Instruction: Name the home team for carlton away team ### Input: CREATE TABLE table_name_77 (\n    home_team VARCHAR,\n    away_team VARCHAR\n) ### Response: SELECT home_team FROM table_name_77 WHERE away_team = "carlton"'}

# Load Gemma 2

In [6]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

## Prompt Format
```
<bos><start_of_turn>user
Write a hello world program<end_of_turn>
<start_of_turn>model
```

In [14]:
# Define your input prompt
prompt = "Name the home team for carlton away team.\nCREATE TABLE table_name_77 ( home_team VARCHAR, away_team VARCHAR )\n"
prompt = "Which silver has a Gold smaller than 12, a Rank smaller than 5, and a Bronze of 5?\nCREATE TABLE table_name_24 ( silver VARCHAR, bronze VARCHAR, gold VARCHAR, rank VARCHAR )"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# Generate a response from the model
output = model.generate(input_ids, max_length=50, num_return_sequences=1)

# Decode the generated tokens to text
response = tokenizer.decode(output[0], skip_special_tokens=True)

# Print the model's response
print(response)

Which silver has a Gold smaller than 12, a Rank smaller than 5, and a Bronze of 5?
CREATE TABLE table_name_24 ( silver VARCHAR, bronze VARCHAR, gold VARCHAR, rank VARCHAR )
VALUES
